In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.image import imread
from keras import datasets, layers, models
import tensorflow as tf

import numpy as np
!pip install utils
import utils

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

In [2]:
train_sample=pd.read_csv('train.csv')
test_sample=pd.read_csv('test.csv')
solution = pd.read_csv('solution_format.csv')

In [3]:
train_sample.head(10)

,labels,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,...,f_1190,f_1191,f_1192,f_1193,f_1194,f_1195,f_1196,f_1197,f_1198,f_1199
0,0,-2.033875,0.978446,-0.142131,-0.177117,-1.470684,1.669562,-0.196530,-0.125239,-0.452284,...,-1.111266,0.716084,0.060039,0.301279,-1.174846,-1.076498,-0.069452,-0.604012,-2.179176,0.558003
1,1,-0.348835,0.294815,-0.557577,-2.020773,-1.234715,1.633930,-1.680658,-0.358146,0.166122,...,0.735240,0.829781,1.521941,1.347946,0.754505,1.330642,-0.754453,0.582956,0.252671,1.495870
2,1,0.113248,-0.607726,-0.947791,0.830851,0.998291,0.498321,-1.493958,0.789572,-1.311018,...,0.104698,0.616189,-1.035953,2.111387,-0.984415,1.148076,-1.433554,0.243372,0.170083,1.274795
3,0,1.223321,-0.479048,-1.925789,1.680377,0.021840,-1.453307,0.605559,-0.019024,1.065448,...,0.360237,-1.957863,-0.123384,1.505329,0.660290,-1.769443,-0.547756,-0.568122,0.244645,0.982116
4,0,0.160109,0.422684,-0.308029,0.227744,0.432854,0.608348,0.193832,1.035091,-0.538868,...,0.416629,1.441766,0.212572,-0.994721,1.143999,-2.166923,-1.199248,-1.028636,0.752791,0.317169
5,0,-0.526642,-0.590432,0.328210,-0.547433,-0.692556,0.175954,0.539330,0.578212,0.547327,...,1.131889,0.250693,-0.215257,0.236730,0.905414,-0.102015,-2.453250,-1.562715,-0.324501,1.590111
6,0,-0.199279,0.831787,-1.052687,0.491722,-0.571469,-1.449519,-0.227167,0.451431,0.090848,...,1.081787,0.875338,-1.217201,-0.297249,-0.133911,-0.865070,-0.128914,1.153154,0.363864,0.670339
7,0,-0.906233,0.987997,-2.056229,1.078932,0.501608,0.826041,-0.136533,-0.931548,-0.657145,...,1.041859,-0.313667,1.492701,-0.425727,-0.425587,-0.408214,2.041689,0.814182,0.211569,0.343013
8,0,-0.283658,0.964051,-0.349123,-0.162445,0.238172,0.501472,0.413006,-1.183602,-0.221886,...,-1.612677,-0.714356,-0.033445,-2.350722,-1.373337,-1.521727,0.015327,0.114279,1.320421,0.979955
9,0,1.382773,-2.415394,0.969006,-1.690371,1.509442,0.914959,-0.161389,0.952598,-0.044989,...,0.875176,-0.855081,0.318588,0.252447,0.400645,1.732217,-0.390191,0.628386,-1.518295,-0.218986


In [4]:
X_train=train_sample.drop('labels',axis ='columns')
Y_train=train_sample['labels']
X_test = test_sample.drop('id',axis ='columns')
Y_test = solution['labels']

In [5]:
Y_test.head()

0    0
1    1
2    0
3    1
4    0
Name: labels, dtype: int64

In [6]:
X_train.head()

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_1190,f_1191,f_1192,f_1193,f_1194,f_1195,f_1196,f_1197,f_1198,f_1199
0,-2.033875,0.978446,-0.142131,-0.177117,-1.470684,1.669562,-0.196530,-0.125239,-0.452284,-0.128052,...,-1.111266,0.716084,0.060039,0.301279,-1.174846,-1.076498,-0.069452,-0.604012,-2.179176,0.558003
1,-0.348835,0.294815,-0.557577,-2.020773,-1.234715,1.633930,-1.680658,-0.358146,0.166122,-1.656990,...,0.735240,0.829781,1.521941,1.347946,0.754505,1.330642,-0.754453,0.582956,0.252671,1.495870
2,0.113248,-0.607726,-0.947791,0.830851,0.998291,0.498321,-1.493958,0.789572,-1.311018,0.848524,...,0.104698,0.616189,-1.035953,2.111387,-0.984415,1.148076,-1.433554,0.243372,0.170083,1.274795
3,1.223321,-0.479048,-1.925789,1.680377,0.021840,-1.453307,0.605559,-0.019024,1.065448,0.717341,...,0.360237,-1.957863,-0.123384,1.505329,0.660290,-1.769443,-0.547756,-0.568122,0.244645,0.982116
4,0.160109,0.422684,-0.308029,0.227744,0.432854,0.608348,0.193832,1.035091,-0.538868,0.778445,...,0.416629,1.441766,0.212572,-0.994721,1.143999,-2.166923,-1.199248,-1.028636,0.752791,0.317169


In [7]:
X_train = X_train.values.reshape(len(X_train),20,20,3)

Y_train = Y_train.values.reshape(len(Y_train), 1)

X_test = X_test.values.reshape(len(X_test), 20, 20, 3)
Y_test = Y_test.values.reshape(len(Y_test), 1)

X_train = X_train/255.0
X_test = X_test/255.0


In [8]:
print("Shape of X_train: ", X_train.shape)
print("Shape of Y_train: ", Y_train.shape)
print("Shape of X_test: ", X_test.shape)
print("Shape of Y_test: ", Y_test.shape)

Shape of X_train:  (5250, 20, 20, 3)
Shape of Y_train:  (5250, 1)
Shape of X_test:  (2250, 20, 20, 3)
Shape of Y_test:  (2250, 1)


In [9]:
model = models.Sequential([
        layers.Conv2D(filters=16,kernel_size=(11,11),activation='relu',input_shape=(20,20,3),strides=(2,2),use_bias=True),
        layers.Dropout(0.2),
        layers.MaxPool2D((2,2),padding='same'),
        layers.Conv2D(filters=32,kernel_size=(3,3),activation='relu',use_bias=True),
        layers.MaxPool2D((2,2),padding ='same'),

        layers.Flatten(),
        layers.Dense(128,activation='relu',kernel_regularizer=tf.keras.regularizers.L1(l=0.01)),
        layers.Dropout(0.4),
        layers.Dense(32,activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(2,activation='softmax')
        ])

opt =tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=opt,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy',]
    )
model.fit(X_train,Y_train,epochs=10)
accuracy_mod=model.evaluate(X_test,Y_test)

Epoch 1/10
165/165 [==============================] - 2s 8ms/step - loss: 2.1529 - accuracy: 0.7326
Epoch 2/10
165/165 [==============================] - 1s 8ms/step - loss: 0.5296 - accuracy: 0.7446
Epoch 3/10
165/165 [==============================] - 1s 6ms/step - loss: 0.4457 - accuracy: 0.8194
Epoch 4/10
165/165 [==============================] - 1s 6ms/step - loss: 0.4134 - accuracy: 0.8446
Epoch 5/10
165/165 [==============================] - 1s 8ms/step - loss: 0.3942 - accuracy: 0.8512
Epoch 6/10
165/165 [==============================] - 1s 9ms/step - loss: 0.3735 - accuracy: 0.8539
Epoch 7/10
165/165 [==============================] - 1s 8ms/step - loss: 0.3569 - accuracy: 0.8655
Epoch 8/10
165/165 [==============================] - 1s 8ms/step - loss: 0.3402 - accuracy: 0.8726
Epoch 9/10
165/165 [==============================] - 1s 8ms/step - loss: 0.3252 - accuracy: 0.8790
Epoch 10/10
71/71 [==============================] - 0s 2ms/step - loss: 2.2303 - accuracy: 0.5027


In [10]:
xg_model=XGBClassifier(n_estimators=1000)
X_train_flatten = X_train.reshape(X_train.shape[0], -1)
X_test_flatten = X_test.reshape(X_test.shape[0], -1)
xg_model.fit(X_train_flatten,Y_train)
xg_model.fit(X_train_flatten,Y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [11]:
predictions = xg_model.predict(X_test_flatten)

from sklearn.metrics import mean_absolute_error
print("Mean Absolute Error : " + str(mean_absolute_error(predictions, Y_test)))

Mean Absolute Error : 0.5111111111111111


In [12]:
from sklearn.metrics import f1_score
f1_score=(Y_test,predictions)
from sklearn.metrics import classification_report,confusion_matrix

print(classification_report(Y_test,predictions))

              precision    recall  f1-score   support

           0       0.48      0.82      0.61      1085
           1       0.52      0.18      0.27      1165

    accuracy                           0.49      2250
   macro avg       0.50      0.50      0.44      2250
weighted avg       0.50      0.49      0.43      2250

